In [32]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.types import interrupt 
from langgraph.types import interrupt, Command

llm = init_chat_model("openai:gpt-4o-mini")

conn = sqlite3.connect(
    "memory.db",
    check_same_thread=False,)

config = {
    "configurable": {
        "thread_id": "1",
    },
}


In [33]:
class State(MessagesState):
    custom_stuff: str


graph_builder = StateGraph(State)

In [34]:
@tool
def get_human_feedback(poem:str):
    """
    Ask the user for feedback on the poem.
    Use this before returning the final response.
    """
    feedback = interrupt(f"what is your opinion on this poem \n{poem}?")
    return feedback

llm_with_tools = llm.bind_tools(
    tools=[
        get_human_feedback,
        ]
    )

def chatbot(state: State):
    response = llm_with_tools.invoke(
        f"""
        You are an expert in making poems.

        Use the `get_human_feedback` tool to get feedback on your poem.

        Only after you receive positive feedback you can return the final poem.

        ALWAYS ASK FOR FEEDBACK FIRST.

        Here is the conversation history:

        {state["messages"]}
    """
    )
    return {
        "messages": [response]
    }


tool_node = ToolNode(
    tools=[
        get_human_feedback,
    ],
)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn),
)

In [35]:
result = graph.invoke(
    {
        "messages": [
        {"role": "user", "content": "Please write a poem about life."}
        ]
    },
    config=config,
    )

In [36]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Please write a poem about life.
================================== Ai Message ==================================
Tool Calls:
  get_human_feedback (call_60eU5De3sFylwW1qCfPzRRaO)
 Call ID: call_60eU5De3sFylwW1qCfPzRRaO
  Args:
    poem: In the tapestry of time, we weave,
Moments shared, and dreams we believe.
With every sunrise, a chance to be,
Adventurers in this grand odyssey.

Paths intertwine, like rivers they flow,
Joy and sorrow, the seeds that we sow.
Through laughter and tears, we learn and grow,
In the dance of life, let your spirit glow.

Seasons change, and so do we,
From budding leaves to the ancient tree.
Each step a story, a lesson to share,
In the embrace of life, find beauty rare.

So cherish the now, let your heart be free,
Life is a canvas, paint your decree.
In the cycle of living, find your own song,
For in this vast journey, where we all belong.


In [37]:
snapshot = graph.get_state(config)

snapshot.next

('tools',)

In [38]:
response = Command(resume="It looks great!")

result = graph.invoke(
    response,
    config=config,
)
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Please write a poem about life.
================================== Ai Message ==================================
Tool Calls:
  get_human_feedback (call_60eU5De3sFylwW1qCfPzRRaO)
 Call ID: call_60eU5De3sFylwW1qCfPzRRaO
  Args:
    poem: In the tapestry of time, we weave,
Moments shared, and dreams we believe.
With every sunrise, a chance to be,
Adventurers in this grand odyssey.

Paths intertwine, like rivers they flow,
Joy and sorrow, the seeds that we sow.
Through laughter and tears, we learn and grow,
In the dance of life, let your spirit glow.

Seasons change, and so do we,
From budding leaves to the ancient tree.
Each step a story, a lesson to share,
In the embrace of life, find beauty rare.

So cherish the now, let your heart be free,
Life is a canvas, paint your decree.
In the cycle of living, find your own song,
For in this vast journey, where we all belong.
================================= Tool M

In [39]:
snapshot = graph.get_state(config)

snapshot.next

()